<a href="https://colab.research.google.com/github/julienif/julifsChessBot/blob/main/julifsChessBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip3 install torchrl
!pip3 install gym[mujoco]
!pip3 install gymnasium
!pip3 install tqdm
!pip3 install chess

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 5.2 MB/s eta 0:00:00


In [19]:
# imports

In [29]:
import warnings
warnings.filterwarnings("ignore")
from torch import multiprocessing

import numpy as np
import chess
import torch
import gymnasium as gym
from torch import nn
from tensordict.nn import TensorDictModule
from tensordict.nn.distributions import NormalParamExtractor
from gym import spaces

In [30]:
# chess RL environment

In [31]:
class ChessEnv(gym.Env):
  def __init__(self, color_trained='white'):
    super(ChessEnv, self).__init__()
    self.color_trained = color_trained
    self.board = chess.Board()
    self.action_space = spaces.Discrete(4096) # assume max 4096 legal moves
    self.observation_space = spaces.Box(low=0, high=1, shape=(8, 8, 13))
    self.reward_range = (-1, 1)

  def reset(self):
    self.board.reset()
    return self._get_observation, {}

  def step(self, action):
    self.board.push(chess.Move.from_uci(action))

    obs = self._get_observation()
    reward = self._get_reward()
    terminated = self.board.is_game_over()
    truncated = False
    info = {}

    return obs, reward, terminated, truncated, info

  def render(self):
    print(self.board)
    return None

  def close(self):
    pass

  def _get_observation(self):
    board_state = np.zeros((8, 8, 13), dtype=np.float32)

    piece_map = {
      'p': 0, 'n': 1, 'b': 2, 'r': 3, 'q': 4, 'k': 5,
      'P': 6, 'N': 7, 'B': 8, 'R': 9, 'Q': 10, 'K': 11,
      '.': 12
    }

    for square in chess.SQUARES:
      piece = self.board.piece_at(square)
      if piece is not None:
        idx = piece_map[piece.symbol()]
        board_state[chess.square_rank(square), chess.square_file(square), idx] = 1
      else:
        board_state[chess.square_rank(square), chess.square_file(square), 12] = 1

    return board_state

  def _get_reward(self):
    # might change this as we would have a constant negative reward if the bot is lead in the game
    reward = 0.0
    if self.board.is_checkmate():
      if self.board.turn == chess.WHITE: # black won
        reward = -1.0
      else: # white won
        reward = 1.0
    elif self.board.is_stalemate() or self.board.is_insufficient_material() or self.board.is_seventyfive_moves():
      reward = 0.0
    else:
      white_score = sum([piece.value for piece in self.board.piece_map().values() if piece.color == chess.WHITE])
      black_score = sum([piece.value for piece in self.board.piece_map().values() if piece.color == chess.BLACK])

      material_adv = white_score - black_score
      norm_material_adv = material_adv / 39.0 # 39 is the max mat diff

      total_moves = len(list(board.move_stack))
      progress = min(total_moves / 150.0, 1.0) # 150 moves is the average game

      reward = 0.7 * norm_material_adv + 0.3 * progress

    return reward if self.color_trained == 'white' else -reward

In [32]:
# implement training